In [2]:
import os, sys
import pandas as pd
from tqdm import tqdm
import requests
import json
from pandas.io.json import json_normalize
sys.path.insert(0,'/home/nuria/workspace/utils3/lib/')
import abravo_lib as utils

In [3]:
# api addresses
biolink = 'https://api.monarchinitiative.org/api'

# endpoint
endpoint = '/association'

# path to data
path = os.getcwd() + '/find_associations'
if not os.path.isdir(path): os.makedirs(path)

# read data
sys.path.insert(0, './find_associations/')
#/workspace/repurposing/rephetio-su/ngly1-net/networks/curated_1shellMonarchExpansion/data

In [16]:
# read ngly1 network nodes list
# ngly1 network

In [17]:
# get associations from Monarch: api
nodes_l = ['NCBIGene:55768'] * 1

# Monarch edges - compact version
data = {'use_compact_associations':'true','fl_excludes_evidence':'true'}
for node in tqdm(nodes_l, total = len(nodes_l)):
    # out edges: from/
    rc_out = requests.get('{}{}/from/{}'.format(biolink,endpoint,node),data)

    # in edges: to/
    rc_in = requests.get('{}{}/to/{}'.format(biolink,endpoint,node),data)

100%|██████████| 1/1 [00:00<00:00,  2.29it/s]


In [18]:
rc_out.json()

{'associations': None,
 'compact_associations': [{'objects': ['GO:0046872', 'GO:0005515'],
   'relation': 'RO:0002327',
   'subject': 'NCBIGene:55768'},
  {'objects': ['GO:0006457',
    'KEGG-path:map04141',
    'GO:0006517',
    'REACT:R-HSA-532668'],
   'relation': 'RO:0002331',
   'subject': 'NCBIGene:55768'},
  {'objects': ['OMIM:615273'], 'relation': None, 'subject': 'NCBIGene:55768'},
  {'objects': ['GO:0005737', 'GO:0005634', 'GO:0005829'],
   'relation': 'BFO:0000050',
   'subject': 'NCBIGene:55768'}],
 'facet_counts': {},
 'facet_pivot': None,
 'numFound': None,
 'objects': None,
 'start': None}

In [19]:
# save the result
with open('{}/{}_out_compact_associations.json'.format(path, node), 'w') as f:
    json.dump(rc_out.json(), f, sort_keys=True, indent=4)
with open('{}/{}_in_compact_associations.json'.format(path, node), 'w') as f:
    json.dump(rc_in.json(), f, sort_keys=True, indent=4)

In [20]:
rc_out_df = json_normalize(rc_out.json(), 'compact_associations')
rc_out_df.head(2)
rc_out_df = rc_out_df.rename(
                     columns= {
                         'subject': 'node_queried_id',
                         'relation':'relation_id',
                         'objects': 'node_associated_id'
                     }
)

In [21]:
rc_in_df = json_normalize(rc_in.json(), 'compact_associations')
rc_in_df.head(2)
rc_in_df = rc_in_df.rename(
                     columns= {
                         'objects': 'node_queried_id',
                         'relation':'relation_id',
                         'subject': 'node_associated_id'
                     }
)

In [22]:
associations_df = pd.concat([rc_out_df, rc_in_df], ignore_index=True)
associations_df = associations_df[['node_queried_id', 'relation_id', 'node_associated_id']]
associations_df.to_csv('{}/{}_compact_associations.tsv'.format(path, node), sep='\t', index=False, header=True)
associations_df.head()

,node_queried_id,relation_id,node_associated_id
0,NCBIGene:55768,RO:0002327,"[GO:0046872, GO:0005515]"
1,NCBIGene:55768,RO:0002331,"[GO:0006457, KEGG-path:map04141, GO:0006517, R..."
2,NCBIGene:55768,None,[OMIM:615273]
3,NCBIGene:55768,BFO:0000050,"[GO:0005737, GO:0005634, GO:0005829]"
4,[NCBIGene:55768],None,Coriell:GM25340


In [87]:
# Monarch edges - full version
data = {'fl_excludes_evidence':'true'}
for node in tqdm(nodes_l, total = len(nodes_l)):
    # out edges: from/
    r_out = requests.get('{}{}/from/{}'.format(biolink,endpoint,node),data)

    # in edges: to/
    r_in = requests.get('{}{}/to/{}'.format(biolink,endpoint,node),data)

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [88]:
r_out.json()

{'associations': [{'evidence_graph': {'edges': [{'obj': 'ECO:0000304',
      'pred': 'RO:0002558',
      'sub': 'MONARCH:431a40f1d20afbb9e4c94aabb82bd8f2a7053342'},
     {'obj': 'NCBIGene:55768',
      'pred': 'OBAN:association_has_subject',
      'sub': 'MONARCH:431a40f1d20afbb9e4c94aabb82bd8f2a7053342'},
     {'obj': 'REACT:R-HSA-532668',
      'pred': 'RO:0002331',
      'sub': 'NCBIGene:55768'},
     {'obj': 'REACT:R-HSA-532668',
      'pred': 'OBAN:association_has_object',
      'sub': 'MONARCH:431a40f1d20afbb9e4c94aabb82bd8f2a7053342'}],
    'nodes': [{'id': 'ECO:0000304',
      'lbl': 'traceable author statement used in manual assertion'},
     {'id': 'NCBIGene:55768', 'lbl': 'NGLY1'},
     {'id': 'MONARCH:431a40f1d20afbb9e4c94aabb82bd8f2a7053342', 'lbl': None},
     {'id': 'REACT:R-HSA-532668',
      'lbl': 'N-glycan trimming in the ER and Calnexin/Calreticulin cycle'}]},
   'evidence_types': None,
   'id': '7b9b7ffd-9e85-494e-8f49-e5fbb7b7d508',
   'object': {'categories': Non

In [25]:
# save the result
with open('{}/{}_out_associations.json'.format(path, node), 'w') as f:
    json.dump(r_out.json(), f, sort_keys=True, indent=4)
with open('{}/{}_in_associations.json'.format(path, node), 'w') as f:
    json.dump(r_in.json(), f, sort_keys=True, indent=4)

In [26]:
r_out_df = json_normalize(r_out.json(), 'associations')
r_out_df.head(2)

,evidence_graph,evidence_types,id,object,object_extension,provided_by,publications,qualifiers,relation,slim,subject,subject_extension,type
0,"{'nodes': [{'id': 'ECO:0000304', 'lbl': 'trace...",None,7b9b7ffd-9e85-494e-8f49-e5fbb7b7d508,"{'deprecated': None, 'replaced_by': None, 'lab...",None,[https://data.monarchinitiative.org/ttl/reacto...,None,None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,None
1,{'nodes': [{'id': ':.well-known/genid/person-3...,None,af89e914-68e5-4b3f-8d02-ee6d214a8a19,"{'deprecated': None, 'replaced_by': None, 'lab...",None,[https://data.monarchinitiative.org/ttl/kegg.t...,None,None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,None


In [27]:
r_in_df = json_normalize(r_in.json(), 'associations')
r_in_df.head(2)

,evidence_graph,evidence_types,id,object,object_extension,provided_by,publications,qualifiers,relation,slim,subject,subject_extension,type
0,{'nodes': [{'id': ':.well-known/genid/610661.0...,None,005099b2-7ca9-4c41-a93d-20e02a43c9e7,"{'deprecated': None, 'replaced_by': None, 'lab...",None,[https://data.monarchinitiative.org/ttl/omim.t...,None,None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,None
1,{'nodes': [{'id': ':.well-known/genid/genoGM25...,None,e758883e-b6aa-46a1-a386-0e1f85539cad,"{'deprecated': None, 'replaced_by': None, 'lab...",None,[https://data.monarchinitiative.org/ttl/omim.t...,None,None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,"{'deprecated': None, 'replaced_by': None, 'lab...",None,None


In [28]:
r_in_df = json_normalize(r_in.json()['associations'])
r_in_df.head()

,evidence_graph.edges,evidence_graph.nodes,evidence_types,id,object.categories,object.consider,object.deprecated,object.description,object.id,object.label,...,subject.id,subject.label,subject.replaced_by,subject.synonyms,subject.taxon.id,subject.taxon.label,subject.types,subject.xrefs,subject_extension,type
0,[{'obj': ':.well-known/genid/610661.0004-61066...,[{'id': ':.well-known/genid/610661.0004-610661...,None,005099b2-7ca9-4c41-a93d-20e02a43c9e7,None,None,None,None,NCBIGene:55768,NGLY1,...,:.well-known/genid/person-GM25990,affected female proband with Congenital disord...,None,None,NCBITaxon:9606,Homo sapiens,None,None,None,None
1,"[{'obj': ':.well-known/genid/genoGM25347', 'su...","[{'id': ':.well-known/genid/genoGM25347', 'lbl...",None,e758883e-b6aa-46a1-a386-0e1f85539cad,None,None,None,None,NCBIGene:55768,NGLY1,...,:.well-known/genid/person-3200-1,affected male proband with Congenital disorder...,None,None,NCBITaxon:9606,Homo sapiens,None,None,None,None
2,"[{'obj': 'OMIM:610661.0003', 'sub': ':.well-kn...","[{'id': ':.well-known/genid/610661.0003', 'lbl...",None,a9fa7673-df2a-4c2d-b4a5-0762fbef239d,None,None,None,None,NCBIGene:55768,NGLY1,...,:.well-known/genid/person-3213-1,affected female proband with Congenital disord...,None,None,NCBITaxon:9606,Homo sapiens,None,None,None,None
3,"[{'obj': 'NCBIGene:55768', 'sub': 'OMIM:610661...","[{'id': 'Coriell:GM25340', 'lbl': 'NIGMS-GM253...",None,d26f2779-04c4-4614-836b-c88da38af258,None,None,None,None,NCBIGene:55768,NGLY1,...,:.well-known/genid/person-3199-1,affected male proband with Congenital disorder...,None,None,NCBITaxon:9606,Homo sapiens,None,None,None,None
4,[{'obj': ':.well-known/genid/610661.0004-61066...,[{'id': ':.well-known/genid/610661.0004-610661...,None,1550dd15-3d25-40d7-883f-9f14fc2c554b,None,None,None,None,NCBIGene:55768,NGLY1,...,:.well-known/genid/person-GM25990,affected female proband with Congenital disord...,None,None,NCBITaxon:9606,Homo sapiens,None,None,None,None


In [29]:
r_in_df.dtypes

evidence_graph.edges    object
evidence_graph.nodes    object
evidence_types          object
id                      object
object.categories       object
object.consider         object
object.deprecated       object
object.description      object
object.id               object
object.label            object
object.replaced_by      object
object.synonyms         object
object.taxon.id         object
object.taxon.label      object
object.types            object
object.xrefs            object
object_extension        object
provided_by             object
publications            object
qualifiers              object
relation.categories     object
relation.consider       object
relation.deprecated     object
relation.description    object
relation.id             object
relation.label          object
relation.replaced_by    object
relation.synonyms       object
relation.types          object
slim                    object
subject.categories      object
subject.consider        object
subject.

In [30]:
in_df = r_in_df[['subject.id','subject.label', 'relation.id', 'relation.label', 'object.id', 'object.label']]
in_df

,subject.id,subject.label,relation.id,relation.label,object.id,object.label
0,:.well-known/genid/person-GM25990,affected female proband with Congenital disord...,None,None,NCBIGene:55768,NGLY1
1,:.well-known/genid/person-3200-1,affected male proband with Congenital disorder...,None,None,NCBIGene:55768,NGLY1
2,:.well-known/genid/person-3213-1,affected female proband with Congenital disord...,None,None,NCBIGene:55768,NGLY1
3,:.well-known/genid/person-3199-1,affected male proband with Congenital disorder...,None,None,NCBIGene:55768,NGLY1
4,:.well-known/genid/person-GM25990,affected female proband with Congenital disord...,None,None,NCBIGene:55768,NGLY1
5,:.well-known/genid/person-3199-1,affected male proband with Congenital disorder...,None,None,NCBIGene:55768,NGLY1
6,:.well-known/genid/person-3213-1,affected female proband with Congenital disord...,None,None,NCBIGene:55768,NGLY1
7,:.well-known/genid/person-3200-1,affected male proband with Congenital disorder...,None,None,NCBIGene:55768,NGLY1
8,Coriell:GM25990,NIGMS-GM25990,None,None,NCBIGene:55768,NGLY1
9,Coriell:GM25340,NIGMS-GM25340,None,None,NCBIGene:55768,NGLY1


In [31]:
# save associations file
r_out_df = json_normalize(r_out.json()['associations'])
out_df = r_out_df[['subject.id','subject.label', 'relation.id', 'relation.label', 'object.id', 'object.label']]
out_df = out_df.rename(
                  columns={
                      'subject.id':'node_queried_id',
                      'subject.label':'node_queried_label',
                      'relation.id':'relation_id',
                      'relation.label':'relation_label',
                      'object.id':'node_associated_id',
                      'object.label':'node_associated_label'
                      # extract also type/category and xref
                  }
)
out_df.head(2)

,node_queried_id,node_queried_label,relation_id,relation_label,node_associated_id,node_associated_label
0,NCBIGene:55768,NGLY1,RO:0002331,involved in,REACT:R-HSA-532668,N-glycan trimming in the ER and Calnexin/Calre...
1,NCBIGene:55768,NGLY1,None,None,OMIM:615273,NGLY1-deficiency


In [32]:
r_in_df = json_normalize(r_in.json()['associations'])
in_df = r_in_df[['subject.id','subject.label', 'relation.id', 'relation.label', 'object.id', 'object.label']]
in_df = in_df.rename(
                columns={
                      'subject.id':'node_associated_id',
                      'subject.label':'node_associated_label',
                      'relation.id':'relation_id',
                      'relation.label':'relation_label',
                      'object.id':'node_queried_id',
                      'object.label':'node_queried_label'
                      # extract also type/category and xref
                }
)
in_df.head(2)

,node_associated_id,node_associated_label,relation_id,relation_label,node_queried_id,node_queried_label
0,:.well-known/genid/person-GM25990,affected female proband with Congenital disord...,None,None,NCBIGene:55768,NGLY1
1,:.well-known/genid/person-3200-1,affected male proband with Congenital disorder...,None,None,NCBIGene:55768,NGLY1


In [33]:
associations_df = pd.concat([out_df, in_df], ignore_index=True)
#ssociations_df = associations_df[['node_queried_id', 'relation_id', 'node_associated_id']]
associations_df = (
    associations_df
    [['node_queried_id', 'node_queried_label', 'relation_id', 'relation_label', 'node_associated_id', 'node_associated_label']]
)
associations_df.to_csv('{}/{}_associations.tsv'.format(path,node), sep='\t', index=False, header=True)
associations_df.head(2)

,node_queried_id,node_queried_label,relation_id,relation_label,node_associated_id,node_associated_label
0,NCBIGene:55768,NGLY1,RO:0002331,involved in,REACT:R-HSA-532668,N-glycan trimming in the ER and Calnexin/Calre...
1,NCBIGene:55768,NGLY1,None,None,OMIM:615273,NGLY1-deficiency


In [34]:
#############################################
# match id with network
network_df = pd.DataFrame({'node_associated_id':['REACT:R-HSA-532668','OMIM:615273','GO:0005829']})
connections_df = associations_df.merge(network_df, how='inner')
connections_df

,node_queried_id,node_queried_label,relation_id,relation_label,node_associated_id,node_associated_label
0,NCBIGene:55768,NGLY1,RO:0002331,involved in,REACT:R-HSA-532668,N-glycan trimming in the ER and Calnexin/Calre...
1,NCBIGene:55768,NGLY1,None,None,OMIM:615273,NGLY1-deficiency
2,NCBIGene:55768,NGLY1,BFO:0000050,is part of,GO:0005829,cytosol


In [35]:
# save to file
connections_df.fillna('None').to_csv('{}/{}_associations_in_ngly1_graph.tsv'.format(path,node), sep='\t', index=False, header=True)
#############################################

In [36]:
# get id -> qid
# Wikidata 1:1 iff gene: 1:n.append(qid(protein))
# list of monarch id
associated_id_list = list(associations_df.node_associated_id)
associated_id_list

['REACT:R-HSA-532668',
 'OMIM:615273',
 'KEGG-path:map04141',
 'GO:0005829',
 'GO:0005515',
 'GO:0006457',
 'GO:0005737',
 'GO:0046872',
 'GO:0005634',
 'GO:0006517',
 ':.well-known/genid/person-GM25990',
 ':.well-known/genid/person-3200-1',
 ':.well-known/genid/person-3213-1',
 ':.well-known/genid/person-3199-1',
 ':.well-known/genid/person-GM25990',
 ':.well-known/genid/person-3199-1',
 ':.well-known/genid/person-3213-1',
 ':.well-known/genid/person-3200-1',
 'Coriell:GM25990',
 'Coriell:GM25340']

In [37]:
# wikidata query
wd_type_property_dict = {'pathway':{'REACT':'P3937'}}
node_type = 'pathway'
node_id = 'REACT'
node_code = '"R-HSA-532668"'

query = """SELECT DISTINCT ?id ?item
WHERE
{
  ?item wdt:""" + wd_type_property_dict.get(node_type).get(node_id) + """ ?id .
  values ?id {""" + node_code + """}
}
"""

# wikidata sparql api
# api address:
api = 'https://query.wikidata.org/sparql'

# api call
r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})

# get results
r.json()
print(r.json())
for wditem in r.json()['results']['bindings']:
    id = wditem['id']['value']
    ruri, qid = wditem['item']['value'].rsplit('/', 1)
    print(id, qid)

{'results': {'bindings': [{'id': {'value': 'R-HSA-532668', 'type': 'literal'}, 'item': {'value': 'http://www.wikidata.org/entity/Q29017194', 'type': 'uri'}}]}, 'head': {'vars': ['id', 'item']}}
R-HSA-532668 Q29017194


In [38]:
# wikidata query for gene encoded proteins alternative
wd_type_property_dict = {'gene':{'NCBIGene':'P351'}}
node_type = 'gene'
node_id = 'NCBIGene'
node_code = '"55768"'

query = """SELECT DISTINCT ?id ?item 
WHERE
{
  ?geneItem wdt:""" + wd_type_property_dict.get(node_type).get(node_id) + """ ?id . # Entrez Gene ID
  ?item wdt:P702 ?geneItem . # Encoded by
  values ?id {""" + node_code + """}
}
"""

# wikidata sparql api
# api address:
api = 'https://query.wikidata.org/sparql'

# api call
r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})

# get results
r.json()
print(r.json())
for wditem in r.json()['results']['bindings']:
    id = wditem['id']['value']
    ruri, qid = wditem['item']['value'].rsplit('/', 1)
    print(id, qid)

{'results': {'bindings': [{'id': {'value': '55768', 'type': 'literal'}, 'item': {'value': 'http://www.wikidata.org/entity/Q21119040', 'type': 'uri'}}]}, 'head': {'vars': ['id', 'item']}}
55768 Q21119040


In [39]:
# wikidata query for gene encoded proteins (the same query as above, 
# but gene has a list of properties that one is encodedBY)
# here the input id is the qid for the gene
wd_type_property_dict = {'gene':{'NCBIGene':'P351', 'encodedBy':'P702'}}
node_type = 'gene'
node_id = 'encodedBy'
node_code = '<http://www.wikidata.org/entity/Q18042037>'
#node_code = concat('<http://www.wikidata.org/entity/','Q18042037','>')

query = """SELECT DISTINCT ?id ?item 
WHERE
{
  ?item wdt:""" + wd_type_property_dict.get(node_type).get(node_id) + """ ?id .
  values ?id {""" + node_code + """}
}
"""

# wikidata sparql api
# api address:
api = 'https://query.wikidata.org/sparql'

# api call
r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})

# get results
r.json()
print(r.json())
for wditem in r.json()['results']['bindings']:
    id = wditem['id']['value']
    ruri, qid = wditem['item']['value'].rsplit('/', 1)
    print(id, qid)

{'results': {'bindings': [{'id': {'value': 'http://www.wikidata.org/entity/Q18042037', 'type': 'uri'}, 'item': {'value': 'http://www.wikidata.org/entity/Q21119040', 'type': 'uri'}}]}, 'head': {'vars': ['id', 'item']}}
http://www.wikidata.org/entity/Q18042037 Q21119040


In [40]:
    # are there id to nqid? yes, e.g. gene <http://www.wikidata.org/entity/Q18026920> => 35 proteins
    #SELECT DISTINCT ?id (count(?item) as ?nitem)
    #WHERE
    #{
    #  ?item wdt:P702 ?id .
    #}
    #group by ?id
    #having (?nitem > 1)
    #order by desc(?nitem)
    #limit 10

#SELECT DISTINCT ?id (group_concat(?item; separator='|') as ?items)
SELECT DISTINCT ?id ?item
    WHERE
    {
      ?item wdt:P702 ?id .
      values ?id {<http://www.wikidata.org/entity/Q18026920>}
    }
#group by ?id

SyntaxError: invalid syntax (<ipython-input-40-857d438703fd>, line 2)

In [51]:
# function to map monarch_id to qid EASY: funcions una per les entitats i l'altra pels encoded proteins
class Wikidata:
    '''This class contains wikidata related data/methods'''
    
    def __init__(self):
        self.wdId2PropDict = {}
    
        self.wdId2PropDict['NCBIGene'] = 'P351' # Entrez Gene ID, code is geneid: 8702 *
        self.wdId2PropDict['ZFIN'] = 'P3870' # ZFIN Gene ID, code is ZDB-GENE-030131-7764 *
        self.wdId2PropDict['MGI'] = 'P671' # Mouse Genome Informatics ID, code is MGI:1913276 
        self.wdId2PropDict['UniProt'] = 'P352' # UniProt ID, code is Q9UHM6 *
        self.wdId2PropDict['UMLS'] = 'P2892' # UMLS CUI, code is C0023206 * # NO RESULTS cuz no ID statement
        self.wdId2PropDict['HP'] = 'P3841' # Human Phenotype Ontology ID, code is HP:0002795
        self.wdId2PropDict['FlyBase'] = 'P3852' # FlyBase Gene ID, code is FBgn0015872 *
        self.wdId2PropDict['UBERON'] = 'P1554' # UBERON ID, code is 0000955 *
        self.wdId2PropDict['DOID'] = 'P699' # Disease Ontology ID, code is DOID:0060068
        self.wdId2PropDict['MESH'] = 'P486' # MeSH ID, code is D002277 *
        self.wdId2PropDict['OMIM'] = 'P492' # OMIM ID, code is 608984 *
        self.wdId2PropDict['Orphanet'] = 'P1550' # Orphanet ID, code is 404454 *
        self.wdId2PropDict['NCBITaxon'] = 'P685' # Gene Ontology ID, code is 157 *
        self.wdId2PropDict['REACT'] = 'P3937' # Reactome pathway ID, code is R-HSA-532668 *
        self.wdId2PropDict['KEGG-path'] = 'P665' # KEGG ID, code is map00520 *
        self.wdId2PropDict['CHEBI'] = 'P683' # ChEBI ID, code is 506227 *
        self.wdId2PropDict['SO'] = 'P3986' # Sequence Ontology ID, code is SO:0001587   

def getQid(dictionary = {}, node_id = 'NCBIGene', node_code = '"8702"'):
    '''This functions returns id to qid.'''
    
    # variables
    wdId2PropDict = Wikidata().wdId2PropDict
    
    # wd query
    propertyQid = wdId2PropDict.get(node_id)
    query = """SELECT DISTINCT ?id ?item
    WHERE
    {
      ?item wdt:""" + propertyQid + """ ?id .
      values ?id {""" + node_code + """}
    }
    """
    
    # wikidata sparql api
    # api address:
    api = 'https://query.wikidata.org/sparql'

    # api call
    r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})
    
    # get results: 'idQidDict => {'id' : {'qid':1, 'qid':1}}
    idQidDict = dictionary
    if not idQidDict:
        idQidDict = dict()
    for wditem in r.json()['results']['bindings']:
        id = wditem['id']['value']
        ruri, qid = wditem['item']['value'].rsplit('/', 1)
        idQidDict = utils.add_elem_with_dictionary(idQidDict, id, qid)
        
    
    return idQidDict
    
#idQidDict = getQid(idQidDict)
getQid(idQidDict)

{'8702': {'Q18033227': 1}, 'SO:0001587': {'Q28371788': 1}}

In [22]:
# function to map monarch_id to qid EASY: funcions una per les entitats i l'altra pels encoded proteins
def getQid2(node_type = 'm-protein', node_id = 'MESH', node_code = '"D006023"'):
    '''This functions returns id to qid.'''
    
    # dictionaries {entityType:wd-properties}
    # lack of: MP, CL, CHR,ClinVarVariant, FOODON
    # mesh, umls, chebi,... can be in more than one type of node
    wdId2PropDict = {}
    
    wdId2PropDict['gene'] = {'NCBIGene':'P351'} # Entrez Gene ID, code is geneid: 8702 *
    wdId2PropDict['z-gene'] = {'ZFIN':'P3870'} # ZFIN Gene ID, code is ZDB-GENE-030131-7764 *
    wdId2PropDict['m-gene'] = {'MGI':'P671'} # Mouse Genome Informatics ID, code is MGI:1913276 
    wdId2PropDict['u-protein'] = {'UniProt':'P352'} # UniProt ID, code is Q9UHM6 *
    wdId2PropDict['m-protein'] = {'MESH':'P486'} # MeSH ID, code is D006023 * # NO RESULTS cuz no ID statement
    wdId2PropDict['um-protein'] = {'UMLS':'P2892'} # UMLS CUI, code is C0023206 * # NO RESULTS cuz no ID statement
    wdId2PropDict['h-phenotype'] = {'HP':'P3841'} # Human Phenotype Ontology ID, code is HP:0002795
    wdId2PropDict['f-phenotype'] = {'FlyBase':'P3852'} # FlyBase Gene ID, code is FBgn0015872 *
    wdId2PropDict['anatomy'] = {'UBERON':'P1554'} # UBERON ID, code is 0000955 *
    wdId2PropDict['d-disease'] = {'DOID':'P699'} # Disease Ontology ID, code is DOID:0060068
    wdId2PropDict['m-disease'] = {'MESH':'P486'} # MeSH ID, code is D002277 *
    wdId2PropDict['o-disease'] = {'OMIM':'P492'} # OMIM ID, code is 608984 *
    wdId2PropDict['or-disease'] = {'Orphanet':'P1550'} # Orphanet ID, code is 404454 *
    wdId2PropDict['taxon'] = {'NCBITaxon':'P685'} # Gene Ontology ID, code is 157 *
    wdId2PropDict['r-pathway'] = {'REACT':'P3937'} # Reactome pathway ID, code is R-HSA-532668 *
    wdId2PropDict['k-pathway'] = {'KEGG-path':'P665'} # KEGG ID, code is map00520 *
    wdId2PropDict['chemical'] = {'CHEBI':'P683'} # ChEBI ID, code is 506227 *
    wdId2PropDict['sequence'] = {'SO':'P3986'} # Sequence Ontology ID, code is SO:0001587
    
    # wd query
    propertyQid = wdId2PropDict.get(node_type).get(node_id)
    query = """SELECT DISTINCT ?id ?item
    WHERE
    {
      ?item wdt:""" + propertyQid + """ ?id .
      values ?id {""" + node_code + """}
    }
    """
    
    # wikidata sparql api
    # api address:
    api = 'https://query.wikidata.org/sparql'

    # api call
    r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})
    
    # get results: 'idQidDict => {'id' : {'qid':1, 'qid':1}}
    idQidDict = dict()
    for wditem in r.json()['results']['bindings']:
        id = wditem['id']['value']
        ruri, qid = wditem['item']['value'].rsplit('/', 1)
        idQidDict = utils.add_elem_with_dictionary(idQidDict, id, qid)
        
    
    return idQidDict
    
getQid2()

{}

In [ ]:
# function to map monarch_id to qid complicada: tot en una funcio
def getQid(node_type = 'gene', node_id = 'NCBIGene', node_code = '"8702"'):
    '''This functions returns id to qid.'''
    
    # dictionaries {entityType:wd-properties}
    wd_type_property_dict = {}
    wd_type_property_dict['pathway'] = {'REACT':'P3937'}
    wd_type_property_dict['gene'] = {'NCBIGene':'P351', 'encodedBy':'P702'}
    wd_type_property_dict['anatomy'] = {'UBERON':'P351'}
    
    
    for key1 in wd_type_property_dict:
        if 'NCBIGene' in wd_type_property_dict[key1]:
            print('yes')
        else:
            print('no')
    
    # wd query
    # if node_type is gene -> add uri_namespace to node_code
    wd = '<http://www.wikidata.org/entity/'
    if node_code.startswith('Q'):
        node_code = pd.concat(wd,node_code,'>')
    
    query = """SELECT DISTINCT ?id ?item
    WHERE
    {
      ?item wdt:""" + wd_type_property_dict.get(node_type).get(node_id) + """ ?id .
      values ?id {""" + node_code + """}
    }
    """
    
    # are there id to nqid? yes, e.g. gene <http://www.wikidata.org/entity/Q18026920> => 35 proteins
    #SELECT DISTINCT ?id (count(?item) as ?nitem)
    #WHERE
    #{
    #  ?item wdt:P702 ?id .
    #}
    #group by ?id
    #having (?nitem > 1)
    #order by desc(?nitem)
    #limit 10
    
    
    # wikidata sparql api
    # api address:
    api = 'https://query.wikidata.org/sparql'

    # api call
    r = requests.post(api, data={'query': query}, headers={'Accept':'application/sparql-results+json'})
    
    # get results
    idQid_dct = {}
    for wditem in r.json()['results']['bindings']:
        id = wditem['id']['value']
        ruri, qid = wditem['item']['value'].rsplit('/', 1)
        #idQid[id] = qid
        print(id, qid)
        
    
    #return dict
    
getQid()

In [ ]:
# get {'id' : list(qids)} dict from wikidata
for assoc_id in associated_id_list:
    namespace, nid = assoc_id.split(':')
    if namespace == 'REACT':
        # query wikidata - return a dict {id:list(qid)}

In [118]:
# function get monarch associations
def getMonarchAssociationsResponse(node = 'NCBIGene:55768'):
    '''This function performs api calls to Monarch for in and out edges. 
        It returns associated nodes'''
    
    # api addresses
    biolink = 'https://api.monarchinitiative.org/api/association'
    
    # parameters
    #data = {'use_compact_associations':'true','fl_excludes_evidence':'true'}
    data = {'fl_excludes_evidence':'true'}
    
    # out edges: from/
    r_out = requests.get('{}/from/{}'.format(biolink,node),data)

    # in edges: to/
    r_in = requests.get('{}/to/{}'.format(biolink,node),data)

    return r_out, r_in 

In [119]:
# prepare monarch api response as a dataframe or tuple?
def prepareMonarchAssociationsResponseAsDf(r_out, r_in):
    '''This function prepare the api object response from monarch to a dataframe. 
       It returns a dataframe.
    '''
    
    # r_out response object to a dataframe
    
    r_out_df = json_normalize(r_out.json()['associations'])
    out_df = r_out_df[['subject.id','subject.label', 'relation.id', 'relation.label', 'object.id', 'object.label']]
    out_df = out_df.rename(
                  columns={
                      'subject.id':'node_queried_id',
                      'subject.label':'node_queried_label',
                      'relation.id':'relation_id',
                      'relation.label':'relation_label',
                      'object.id':'node_associated_id',
                      'object.label':'node_associated_label'
                      # extract also type/category and xref
                  }
    )
    out_df.head(2)

    # r_in response object to a dataframe
    r_in_df = json_normalize(r_in.json()['associations'])
    in_df = r_in_df[['subject.id','subject.label', 'relation.id', 'relation.label', 'object.id', 'object.label']]
    in_df = in_df.rename(
                columns={
                      'subject.id':'node_associated_id',
                      'subject.label':'node_associated_label',
                      'relation.id':'relation_id',
                      'relation.label':'relation_label',
                      'object.id':'node_queried_id',
                      'object.label':'node_queried_label'
                      # extract also type/category and xref
                }
    )
    in_df.head(2)
    
    # concat response dataframes
    associations_df = pd.concat([out_df, in_df], ignore_index=True)
 
    return associations_df

In [125]:
# prepare monarch api response as a dataframe or tuple?
def prepareMonarchAssociationsResponseAsTuple(r_out, r_in):
    '''This function prepare the api object response from monarch to a tuple. 
       It returns a tuple of edges: {(s,r,o),(s,r,o),...}.
    '''
    
    # variables
    tripleId = set()
    tripleXref = set()
    tripleType = set()
    tripleCategory = set()

    # r_out response object to a dataframe
    for association in r_out.json()['associations']:
        tripleId.add((association['subject']['id'], association['relation']['id'], association['object']['id']))
        tripleXref.add((association['subject']['xrefs'], association['object']['xrefs']))
        tripleType.add((association['subject']['types'], association['relation']['types'], association['object']['types']))
        tripleCategory.add((association['subject']['categories'], association['relation']['categories'], association['object']['categories']))

    # r_in response object to a dataframe
    for association in r_in.json()['associations']:
        tripleId.add((association['object']['id'], association['relation']['id'], association['subject']['id']))
        tripleXref.add((association['object']['xrefs'], association['subject']['xrefs']))
        tripleType.add((association['object']['types'], association['relation']['types'], association['subject']['types']))
        tripleCategory.add((association['object']['categories'], association['relation']['categories'], association['subject']['categories']))

    # print    
    print(tripleId)
    print(tripleXref)
    print(tripleType)
    print(tripleCategory)
    
    #return tripleId

prepareMonarchAssociationsResponseAsTuple(r_out, r_in)

{('NCBIGene:55768', None, 'OMIM:615273'), ('NCBIGene:55768', None, 'Coriell:GM25990'), ('NCBIGene:55768', None, 'Coriell:GM25340'), ('NCBIGene:55768', 'RO:0002327', 'GO:0046872'), ('NCBIGene:55768', 'RO:0002327', 'GO:0005515'), ('NCBIGene:55768', None, ':.well-known/genid/person-3213-1'), ('NCBIGene:55768', 'BFO:0000050', 'GO:0005634'), ('NCBIGene:55768', 'RO:0002331', 'REACT:R-HSA-532668'), ('NCBIGene:55768', 'BFO:0000050', 'GO:0005829'), ('NCBIGene:55768', 'RO:0002331', 'KEGG-path:map04141'), ('NCBIGene:55768', 'BFO:0000050', 'GO:0005737'), ('NCBIGene:55768', None, ':.well-known/genid/person-3200-1'), ('NCBIGene:55768', None, ':.well-known/genid/person-3199-1'), ('NCBIGene:55768', 'RO:0002331', 'GO:0006517'), ('NCBIGene:55768', None, ':.well-known/genid/person-GM25990'), ('NCBIGene:55768', 'RO:0002331', 'GO:0006457')}
{(None, None)}
{(None, None, None)}
{(None, None, None)}


In [139]:
# prepare monarch api response as a dataframe or tuple?
def prepareMonarchAssociationsResponseAsTupleAlt(r_out, r_in):
    '''This function prepare the api object response from monarch to a tuple. 
       It returns three lists, one for subjects, relations, and objects, 
       where each list is a list of dictionaries where each dict is a node.
    '''
    
    # variables
    sub_l = list()
    rel_l = list()
    obj_l = list()
    tripleId = set()
    tripleXref = set()
    tripleType = set()
    tripleCategory = set()

    # r_out response object to a dataframe
    # list of dictionaries where each dict is a node
    for association in r_out.json()['associations']:
        sub_l.append(association['subject'])
        rel_l.append(association['relation'])
        obj_l.append(association['object'])  

    # r_in response object to a dataframe
    for association in r_in.json()['associations']:
        sub_l.append(association['object'])
        rel_l.append(association['relation'])
        obj_l.append(association['subject'])  
    
    return sub_l, rel_l, obj_l

def edges(sub_l, rel_l, obj_l, attribute = 'id'):
    '''
       Function that builds triples with an attribute for each node. it returns a tuple with the edges
    '''
    edges = set()
    # compose tuple
    for i in range(len(sub_l)):
        sub = sub_l[i][attribute]
        rel = rel_l[i][attribute]
        obj = obj_l[i][attribute]
        edges.add((sub, rel, obj))
        
    return edges

sub_l, rel_l, obj_l = prepareMonarchAssociationsResponseAsTupleAlt(r_out, r_in)
edges = edges(sub_l, rel_l, obj_l, 'types')
print(edges)

{(None, None, None)}


In [ ]:
# function to map monarch_id to qid
def getQid(associations_df):
    '''This functions returns id to qid.'''
    
    # are there id to nqid?
    
    return associations_df

In [ ]:
# function to evaluate the intersection with network nodes
def filtering(nodes, edges):
    """filters all edges down to those which have both nodes in the nodes set"""
    
    keep = set()
    for (start, pred, stop) in edges:
        if {start, stop} <= nodes:
            keep.add((start, pred, stop))
            
    return keep

In [ ]:
# function to evaluate the intersection with network nodes
def getMatch(associations_df, network_df):
    '''This function returns associated nodes that are in the network.'''
    
    # network and associations df have to be in qid 
    
    #network_df = pd.DataFrame({'node_associated_id':['REACT:R-HSA-532668','OMIM:615273','GO:0005829']})
    connections_df = associations_df.merge(network_df, how='inner')
    return connections_df 

In [ ]:
def printFile(path,fileName):
    '''Function to save output to file.'''
    connections_df.fillna('None').to_csv('{}/{}.tsv'.format(path,fileName), sep='\t', index=False, header=True)
    print("File './{}/{}' saved.")

In [ ]:
for node in nodes_l:
    d = getMonarchAssociations()
    e = getIntersection()
    print(d + '_' + e)